In [1]:
import pandas as pd
import numpy as np

start = '20210905'
end = '20211004'

path1 = f'筑波大学 斬桐舞-{start}-{end}.csv'
path2 = f'つくば 斬桐舞-{start}-{end}.csv'
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
dfs = [df1, df2]
# df
# col = df.columns
# col

# 日付データの処理
def convert_date(df, col):
    df = df.copy()
    df[col] = pd.to_datetime(df[col])
    #df['日付'] = df[col].map(lambda x:str(x.year)+str(x.month).zfill(2)+str(x.day).zfill(2))
    df['日付'] = df[col].map(lambda x:str(x.month)+'/'+str(x.day))
    
    #df['月'] = df[col].map(lambda
    # df['日'] = df[col].map(lambda x:x.day)
    df = df.drop(col, axis=1)
    return df

def data_shaping(df):
    df_ = df.copy()
    
    # 不要なカラムを削除
    drops = ["'ID'", "'グループ名'", "'発熱'", "'平熱'", "'最終出勤日'", 
         "'体温(問診票情報)'", "'症状(問診票情報)'", 
         "'通勤手段（選択）'", "'通勤手段（詳細）'",
        "'受診医療機器名'", "'PCR検査（有無）'", 
         "'PCR検査（詳細）'", "'感染者と接触有無'", 
         "'感染者接触（詳細）'", "'県外移動（有無）都道府県外移動、海外渡航 履歴'",
         "'県外移動（詳細：誰が）'", "'県外移動（詳細：場所）'", 
        "'県外移動（期間）'", "'人の過密場所への移動(有無)'", 
         "'人の過密場所への移動（詳細：誰が）'",
        "'人の過密場所への移動（詳細：場所）'", 
         "'人の過密場所への移動(期間表示）'"]
    
    df_ = df_.drop(drops, axis=1)
    
    # カラムの名前を変更
    renames = {"'氏名'":"氏名", 
           "'日付・時刻'":"日付・時刻",
           "'体温(体調情報)'":"体温",
           "'症状(体調情報)'":"症状",
           "'メッセージ'":"メッセージ"}
    df_ = df_.rename(columns=renames)
    
    #日付・時刻→日付
    df_ = convert_date(df_, '日付・時刻')
    
    # 引用符をなくす
    cols = ['氏名', '体温', '症状', 'メッセージ']
    for c in cols:
        df_[c] = df_[c].str[1:]
        df_[c] = df_[c].str[:-1]
        
    return df_


df1 = data_shaping(df1)
df2 = data_shaping(df2)
df2 = df2[df2['氏名'] != '中西 映人']
df2 = df2[df2['氏名'] != '春日 崇伸']
df_ = pd.concat([df1, df2])
# df_

members = [
# 1年生:17
'笹子 恵利',
'大田 小夏',
'牧本 武蔵',
'角 晴香',
'小山 杏奈',
'山地 沙蘭',
'水谷 文音',
'荻 真優子',
'朱 博瑄',
'梶原 城太',
'藤枝 美帆',
'吉永 晴香',
'北野 志保',
'近藤 彩加',
'長谷川 紗希',
'谷渕 晴哉',
'櫻井 俊太郎',

# 2年生:19
'佐久間 悠斗',
'車谷 郁実',
'田嶋 悠楽々',
'青木 日花',
'村崎 咲良',
'鴻巣 遥香',
'米谷 はづき',
'小泉 ときわ',
'天草 萌々',
'高柳 菜月',
'西村 篤朗',
'星野 明日美',
'齊藤 由理',
'佐光 未帆',
'加藤 大輝',
'萩原 怜央',
'花井 菜々子',
'水野 隼',
'寄川 淳聖',
    
# 3年生 :20
'大竹 叶望',
'春日 崇伸',
'片岡 弓澄',
'川邊 笑',
'黒川 夕鶴',
'小林 優太',
'齋藤 真衣子',
'櫻井 藍花里',
'佐藤 綾香',
'佐藤 由',
'塩入 翔也',
'下仲 咲穂',
'島 沙也加',
'田岡 佑基',
'谷口 萌',
'戸井田 風音',
'中西 映人',
'原田 秦冴',
'細谷 なつみ',
'眞鍋 礼子',

# 4年生 :8 
'大坂 実旺',
'小澤 政貴',
'加来 祐子',
'齋藤 亘佑',
'孫 辰希',
'田中 大輔',
'鍋山 雄樹',
'本田 拓海',

# 院生:5
'川端 航平',
'二宮 功伎',
'吉澤 秀俊',
'澁澤 薫',
'若木 良太',

# 社会人さん:21
'赤根 大介',
'赤根 たか子',
'赤根 寛',
'井上 百合子',
'加園 真愛',
'後坊 健太',
'齊藤 憲二',
'齊藤 裕希也',
'佐藤 恵美子',
'鈴木 悠斗',
'中村 雅子',
'中村 公祐',
'坂東 香代子',
'坂東 正樹',
'松本 朔弥',
'安田 清美',
'飯村 良枝',
'土井原 良美',
'宮澤 美幸',
'北條 海織',
'Sesso Bruno',
]
# members_

#名前×日付データ
def name_day(df_, members=members):
    # 名簿
#     members = []
#     for m in df_['氏名'].unique():
#         members.append(m)

    # 日付一覧
    days = []
    for d in df_['日付'].unique():
        days.append(d)

    # http://hxn.blog.jp/archives/11591404.html
    A = pd.DataFrame(index=members, columns=days)

    # 37.5度以上があったら、体温表示、無ければ〇
    for m in members:
        personal_df = df_[df_.氏名==m]
        
        for i in range(len(personal_df)):
            # 高熱の場合は★をつける
            if float(personal_df.iloc[i]['体温']) > 37.4:
                personal_df.iloc[i]['体温'] = '★' + personal_df.iloc[i]['体温']

        for d in A.columns:
            day_data = personal_df[personal_df.日付==d]

            if day_data.empty is True:
                pass

            else:
                text = day_data.iloc[0]['体温']
                A.loc[m][d] = text   
            
    return A

A = name_day(df_)

A.insert(0, '未記録数','0')
#A

a = A.isnull().sum(axis=1)
for i in range(len(a)):
    b = a.iloc[i]
    if b == 0:
        A.iat[i,0] = '〇'
    else:
        A.iat[i,0] = b
#A

# insertは、列には使えるが行には使えないので、
# 転置→insert→転置する必要がある

B = A.copy()
AT = A.T
AT.insert(0, '大学1年', '----')
AT.insert(18, '大学2年', '----')
AT.insert(38, '大学3年', '----')
AT.insert(59, '大学4年', '----')
AT.insert(68, '大学院生', '----')
AT.insert(74, '社会人', '----')
A = AT.T

A1 = A.iloc[:-22]
A2 = A.iloc[-22:]
# A1

term = f'{start}_{end}'
path = f'斬桐舞 体温記録_{term}.xlsx'
# A1.to_excel(path, sheet_name='筑波大学斬桐舞')
# A2.to_excel(path, sheet_name='つくば斬桐舞')

with pd.ExcelWriter(path) as writer:
    A1.to_excel(writer, sheet_name=f"筑波大学 斬桐舞")
    A2.to_excel(writer, sheet_name=f"つくば 斬桐舞")

B1 = B.iloc[:17]
B2 = B.iloc[17:17+19]
B3 = B.iloc[17+19:17+19+20]
B4 = B.iloc[17+19+20:17+19+20+8]
B5 = B.iloc[17+19+20+8:17+19+20+8+5]
B6 = B.iloc[-21:]

def ok_missing(A):  
    OK = []
    MISSING = []

    for i in A.index:
        if A.loc[i].isna().sum() == 0:
            OK.append(i)
        else:
            MISSING.append(i)
    
    return OK, MISSING

O1, M1 = ok_missing(B1)
O2, M2 = ok_missing(B2)
O3, M3 = ok_missing(B3)
O4, M4 = ok_missing(B4)
O5, M5 = ok_missing(B5)
O6, M6 = ok_missing(B6)

def ok_display(O):
    j = 0
    k = []
    for i in O:
        if j == 5:
            print(k)
            j = 0
            k = []

        k.append(i)
        j += 1
    print(k)

print(f'{start}-{end}')
print()
print('筑波大学1年:', len(O1))
ok_display(O1)
print()
print('筑波大学2年:', len(O2))
ok_display(O2)
print()
print('筑波大学3年:', len(O3))
ok_display(O3)
print()
print('筑波大学4年:', len(O4))
ok_display(O4)
print()
print('筑波大学院生:', len(O5))
ok_display(O5)
print()
print('つくば斬桐舞:', len(O6))
ok_display(O6)
print()

20210905-20211004

筑波大学1年: 7
['大田 小夏', '牧本 武蔵', '角 晴香', '小山 杏奈', '水谷 文音']
['朱 博瑄', '長谷川 紗希']

筑波大学2年: 8
['佐久間 悠斗', '村崎 咲良', '天草 萌々', '西村 篤朗', '星野 明日美']
['佐光 未帆', '加藤 大輝', '花井 菜々子']

筑波大学3年: 6
['下仲 咲穂', '戸井田 風音', '中西 映人', '原田 秦冴', '細谷 なつみ']
['眞鍋 礼子']

筑波大学4年: 2
['小澤 政貴', '加来 祐子']

筑波大学院生: 0
[]

つくば斬桐舞: 7
['赤根 大介', '赤根 たか子', '赤根 寛', '齊藤 憲二', '鈴木 悠斗']
['松本 朔弥', '飯村 良枝']



In [3]:
print('出てない人リスト')
print(f'{start}-{end}')
print()
print('筑波大学1年:', len(M1))
ok_display(M1)
print()
print('筑波大学2年:', len(M2))
ok_display(M2)
print()
print('筑波大学3年:', len(M3))
ok_display(M3)
print()
print('筑波大学4年:', len(M4))
ok_display(M4)
print()
print('筑波大学院生:', len(M5))
ok_display(M5)
print()
print('つくば斬桐舞:', len(M6))
ok_display(M6)
print()

出てない人リスト
20210905-20211004

筑波大学1年: 10
['笹子 恵利', '山地 沙蘭', '荻 真優子', '梶原 城太', '藤枝 美帆']
['吉永 晴香', '北野 志保', '近藤 彩加', '谷渕 晴哉', '櫻井 俊太郎']

筑波大学2年: 11
['車谷 郁実', '田嶋 悠楽々', '青木 日花', '鴻巣 遥香', '米谷 はづき']
['小泉 ときわ', '高柳 菜月', '齊藤 由理', '萩原 怜央', '水野 隼']
['寄川 淳聖']

筑波大学3年: 14
['大竹 叶望', '春日 崇伸', '片岡 弓澄', '川邊 笑', '黒川 夕鶴']
['小林 優太', '齋藤 真衣子', '櫻井 藍花里', '佐藤 綾香', '佐藤 由']
['塩入 翔也', '島 沙也加', '田岡 佑基', '谷口 萌']

筑波大学4年: 6
['大坂 実旺', '齋藤 亘佑', '孫 辰希', '田中 大輔', '鍋山 雄樹']
['本田 拓海']

筑波大学院生: 5
['川端 航平', '二宮 功伎', '吉澤 秀俊', '澁澤 薫', '若木 良太']

つくば斬桐舞: 14
['井上 百合子', '加園 真愛', '後坊 健太', '齊藤 裕希也', '佐藤 恵美子']
['中村 雅子', '中村 公祐', '坂東 香代子', '坂東 正樹', '安田 清美']
['土井原 良美', '宮澤 美幸', '北條 海織', 'Sesso Bruno']

